In [1]:
import pandas as pd

In [2]:
# ! wget https://synthetichealth.github.io/synthea-sample-data/downloads/synthea_sample_data_csv_apr2020.zip

--2023-09-07 10:22:52--  https://synthetichealth.github.io/synthea-sample-data/downloads/synthea_sample_data_csv_apr2020.zip
Resolving synthetichealth.github.io (synthetichealth.github.io)... 185.199.108.153, 185.199.110.153, 185.199.109.153, ...
Connecting to synthetichealth.github.io (synthetichealth.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 8982431 (8.6M) [application/zip]
Saving to: ‘synthea_sample_data_csv_apr2020.zip.1’

synthea_sample_data 100%[===================>]   8.57M  2.59MB/s    in 3.4s    

2023-09-07 10:23:00 (2.54 MB/s) - ‘synthea_sample_data_csv_apr2020.zip.1’ saved [8982431/8982431]



In [3]:
# ! unzip synthea_sample_data_csv_apr2020.zip

Archive:  synthea_sample_data_csv_apr2020.zip
replace csv/medications.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

^C


# EDA Syntha data

In [2]:
df = pd.read_csv("csv/observations.csv")
df.head()

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,VALUE,UNITS,TYPE
0,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,8302-2,Body Height,193.3,cm,numeric
1,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,2.0,{score},numeric
2,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,29463-7,Body Weight,87.8,kg,numeric
3,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,39156-5,Body Mass Index,23.5,kg/m2,numeric
4,2012-01-23T17:45:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e88bc3a9-007c-405e-aabc-792a38f4aa2b,8462-4,Diastolic Blood Pressure,82.0,mm[Hg],numeric


In [8]:
df[df.DESCRIPTION.str.contains("T4")].VALUE.unique()

array(['0.3', '0.2'], dtype=object)

In [9]:
df[df.DESCRIPTION.str.contains("gravity")].VALUE.unique()

array(['1.0'], dtype=object)

In [20]:
df[df.DESCRIPTION.str.contains("istribution")][["DESCRIPTION", "CODE"]].DESCRIPTION.unique()

array(['Erythrocyte distribution width [Entitic volume] by Automated count',
       'Platelet distribution width [Entitic volume] in Blood by Automated count',
       'RDW - Erythrocyte distribution width Auto (RBC) [Entitic vol]'],
      dtype=object)

In [5]:
df.PATIENT.value_counts().describe().round(0).astype(int)

count    1171
mean      256
std       501
min        34
25%       134
50%       164
75%       244
max      9023
Name: PATIENT, dtype: int64

In [6]:
df[df.PATIENT==df.PATIENT.unique()[1]][["DESCRIPTION", "VALUE", "UNITS"]][:30].values[24]

array(['Tobacco smoking status NHIS', 'Never smoker', nan], dtype=object)

In [7]:
observations = ", ".join([" ".join(el) for el in df[df.PATIENT==df.PATIENT.unique()[1]][["DESCRIPTION", "VALUE", "UNITS"]][:30].values[:24]])

In [22]:
len(observations)

1133

In [24]:
observations

'Body temperature 37.1 Cel, Body Height 162.5 cm, Pain severity - 0-10 verbal numeric rating [Score] - Reported 3.0 {score}, Body Weight 77.4 kg, Body Mass Index 29.3 kg/m2, Diastolic Blood Pressure 87.0 mm[Hg], Systolic Blood Pressure 127.0 mm[Hg], Heart rate 70.0 /min, Respiratory rate 13.0 /min, Total Cholesterol 174.1 mg/dL, Triglycerides 101.8 mg/dL, Low Density Lipoprotein Cholesterol 86.9 mg/dL, High Density Lipoprotein Cholesterol 66.9 mg/dL, Leukocytes [#/volume] in Blood by Automated count 4.6 10*3/uL, Erythrocytes [#/volume] in Blood by Automated count 4.2 10*6/uL, Hemoglobin [Mass/volume] in Blood 12.3 g/dL, Hematocrit [Volume Fraction] of Blood by Automated count 45.0 %, MCV [Entitic volume] by Automated count 86.3 fL, MCH [Entitic mass] by Automated count 32.4 pg, MCHC [Mass/volume] by Automated count 33.9 g/dL, Erythrocyte distribution width [Entitic volume] by Automated count 45.7 fL, Platelets [#/volume] in Blood by Automated count 259.0 10*3/uL, Platelet distribution 

In [20]:
conditions = pd.read_csv("csv/conditions.csv")
conditions.head(1)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2001-05-01,NaN,1d604da9-9a81-4ba9-80c2-de3375d59b40,8f104aa7-4ca9-4473-885a-bba2437df588,40055000,Chronic sinusitis (disorder)


In [26]:
conditions.DESCRIPTION.unique()

array(['Chronic sinusitis (disorder)', 'Viral sinusitis (disorder)',
       'Acute viral pharyngitis (disorder)',
       'Acute bronchitis (disorder)',
       'Streptococcal sore throat (disorder)', 'Laceration of foot',
       'Acute bacterial sinusitis (disorder)', 'Sprain of wrist',
       'Otitis media', 'Body mass index 30+ - obesity (finding)',
       'Cystitis', 'Fracture of forearm', 'Impacted molars',
       'Normal pregnancy', 'Sinusitis (disorder)', 'Anemia (disorder)',
       'Miscarriage in first trimester',
       'Fetus with unknown complication', 'Prediabetes', 'Hypertension',
       'Escherichia coli urinary tract infection',
       'Whiplash injury to neck', 'Atrial Fibrillation',
       'Sprain of ankle', 'Laceration of forearm',
       'Malignant neoplasm of breast (disorder)', 'Laceration of thigh',
       'Osteoarthritis of knee', 'Preeclampsia',
       'Perennial allergic rhinitis with seasonal variation',
       'Fracture of clavicle', 'Childhood asthma', 'Facia

In [19]:
medications = pd.read_csv("csv/medications.csv")
medications.head(1)

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,2010-05-05T00:26:23Z,2011-04-30T00:26:23Z,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,1e0d6b0e-1711-4a25-99f9-b1c700c9b260,389221,Etonogestrel 68 MG Drug Implant,677.08,0.0,12,8124.96,NaN,NaN


In [18]:
careplans = pd.read_csv("csv/careplans.csv")
careplans.head(1)

,Id,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,d2500b8c-e830-433a-8b9d-368d30741520,2010-01-23,2012-01-23,034e9e3b-2def-4559-bb2a-7850888ae060,d0c40d10-8d87-447e-836e-99d26ad52ea5,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)


In [17]:
devices = pd.read_csv("csv/devices.csv")
devices.head(1)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,UDI
0,2001-07-04T08:42:44Z,NaN,d49f748f-928d-40e8-92c8-73e4c5679711,2500b8bd-dc98-44ef-a252-22dc4f81d61b,72506001,Implantable defibrillator device (physical ob...,(01)67677988606464(11)010613(17)260628(10)2882...


In [16]:
encounters = pd.read_csv("csv/encounters.csv")
encounters.head(1)

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,d0c40d10-8d87-447e-836e-99d26ad52ea5,2010-01-23T17:45:28Z,2010-01-23T18:10:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e002090d-4e92-300e-b41e-7d1f21dee4c6,e6283e46-fd81-3611-9459-0edb1c3da357,6e2f1a2d-27bd-3701-8d08-dae202c58632,ambulatory,185345009,Encounter for symptom,129.16,129.16,54.16,10509002.0,Acute bronchitis (disorder)


In [15]:
imaging = pd.read_csv("csv/imaging_studies.csv")
imaging.head(1)

,Id,DATE,PATIENT,ENCOUNTER,BODYSITE_CODE,BODYSITE_DESCRIPTION,MODALITY_CODE,MODALITY_DESCRIPTION,SOP_CODE,SOP_DESCRIPTION
0,d3e49b38-7634-4416-879d-7bc68bf3e7df,2014-07-08T15:35:36Z,b58731cc-2d8b-4c2d-b327-4cab771af3ef,3a36836d-da25-4e73-808b-972b669b7e4e,40983000,Arm,DX,Digital Radiography,1.2.840.10008.5.1.4.1.1.1.1,Digital X-Ray Image Storage


In [21]:
immunizations = pd.read_csv("csv/immunizations.csv")
immunizations.head(1)

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST
0,2010-07-27T12:58:08Z,10339b10-3cd1-4ac3-ac13-ec26728cb592,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,140,Influenza seasonal injectable preservative ...,140.52


# Clarifai LLM

In [81]:
CLARIFAI_PAT="" # Place your Clarifai Personal Access Token here

In [82]:
import os
import streamlit as st
from langchain.llms import Clarifai
from langchain import PromptTemplate, LLMChain

# https://python.langchain.com/docs/integrations/llms/clarifai
clarifai_llm = Clarifai(
    pat=CLARIFAI_PAT, user_id='openai', app_id='chat-completion', model_id='GPT-3_5-turbo'
)

template = """A patient has the following list of measurements: {observations}

The following measurements are out of normal range: """

prompt = PromptTemplate(template=template, input_variables=["observations"])

In [83]:
# Create LLM chain
llm_chain = LLMChain(prompt=prompt, llm=clarifai_llm)

In [84]:
result = llm_chain.run(observations)

In [85]:
print(result)

- Body Mass Index: 29.3 kg/m2 (overweight)
- Diastolic Blood Pressure: 87.0 mm[Hg] (elevated)
- Total Cholesterol: 174.1 mg/dL (borderline high)
- Low Density Lipoprotein Cholesterol: 86.9 mg/dL (borderline high)
- Erythrocyte distribution width: 45.7 fL (high)
- Platelet distribution width: 440.7 fL (high)
